> 对唐诗三百首进行文字处理，并作为数据集，用RNN尝试文
字生成。

In [22]:
import pandas as pd
import numpy as np
import torch
from torch.nn.utils.rnn import pad_sequence
import jieba
from collections import Counter


In [ ]:
with open('唐诗三百首.txt', 'r', encoding='utf-8') as f:
    text = f.readlines()

poems = [line.strip() for line in text]

tokenized_poems = []
for poem in poems:
    tokens = list(jieba.cut(poem))
    tokenized_poems.append(tokens)



[['琴书', '中有', '得', '，', '衣食', '外', '何求', '。'],
 ['初疑', '白莲花', '，', '浮出', '龙', '王宫', '。'],
 ['难', '为', '此时', '别', '，', '欲别', '愿人留', '。'],
 ['遥知', '太平', '代', '，', '国宝', '在', '名', '都', '。'],
 ['终南山', '北面', '，', '直下', '是', '长安', '。'],
 ['昔为', '同恨', '客', '，', '今为', '独笑', '人', '。'],
 ['此地', '别', '夫子', '，', '今来思', '旧游', '。'],
 ['少结', '相思', '恨', '，', '佳期', '芳草', '前', '。'],
 ['虽无南', '去', '雁', '，', '看取', '北来', '鱼', '。'],
 ['秋亭', '病', '客眠', '，', '庭树', '满枝', '蝉', '。'],
 ['青山', '春暮见', '，', '流水', '夜深', '闻', '。'],
 ['主人', '常不在', '，', '春物', '为', '谁', '开', '。'],
 ['春尽', '有', '归', '日', '，', '老来', '无去', '时', '。'],
 ['前山', '依旧', '碧', '，', '闲草', '经秋绿', '。'],
 ['南中', '多古事', '，', '咏', '遍始', '应', '还', '。'],
 ['琴曲', '唯留古', '，', '书', '多半', '是', '经', '。'],
 ['新居', '未曾', '到', '，', '邻里', '是', '谁家', '。'],
 ['莺', '飞绮阁', '曙', '，', '柳', '拂', '画堂', '春', '。'],
 ['南北', '惟堪恨', '，', '东西', '实可', '嗟', '。'],
 ['岁晚', '青山', '路', '，', '白', '首期', '同归', '。'],
 ['问人', '寻野寺', '，', '牵马渡', '危桥', '。'],
 ['昔闻', '洞庭', '水', '，', '今上', '岳阳

In [29]:
word_counters = Counter()
for poem in tokenized_poems:
    word_counters.update(poem)
word_counters

vocab = ['<PAD>', '<START>', '<END>', '<UNK>'] + [i for i in word_counters.keys() if word_counters[i] >= 3]  # 出现次数大于 3 的词

word2idx = {word: idx for idx, word in enumerate(vocab)}
idx2word = {idx: word for idx, word in enumerate(vocab)}

print(idx2word)

def tokenize(text):
    """将句子转换为词语索引序列"""
    return [word2idx.get(word, word2idx['<UNK>']) for word in text]

def detokenize(indices):
    """将索引序列还原为字符串"""
    return ''.join([idx2word.get(idx, '<UNK>') for idx in indices])


{0: '<PAD>', 1: '<START>', 2: '<END>', 3: '<UNK>', 4: '琴书', 5: '中有', 6: '得', 7: '，', 8: '衣食', 9: '外', 10: '何求', 11: '。', 12: '初疑', 13: '龙', 14: '王宫', 15: '难', 16: '为', 17: '此时', 18: '别', 19: '欲别', 20: '遥知', 21: '太平', 22: '代', 23: '国宝', 24: '在', 25: '名', 26: '都', 27: '终南山', 28: '北面', 29: '直下', 30: '是', 31: '长安', 32: '昔为', 33: '客', 34: '今为', 35: '独笑', 36: '人', 37: '此地', 38: '夫子', 39: '旧游', 40: '相思', 41: '恨', 42: '佳期', 43: '芳草', 44: '前', 45: '去', 46: '雁', 47: '看取', 48: '北来', 49: '鱼', 50: '病', 51: '庭树', 52: '满枝', 53: '蝉', 54: '青山', 55: '流水', 56: '夜深', 57: '闻', 58: '主人', 59: '春物', 60: '谁', 61: '开', 62: '春尽', 63: '有', 64: '归', 65: '日', 66: '老来', 67: '无去', 68: '时', 69: '前山', 70: '依旧', 71: '碧', 72: '咏', 73: '应', 74: '还', 75: '书', 76: '经', 77: '新居', 78: '未曾', 79: '到', 80: '邻里', 81: '谁家', 82: '莺', 83: '曙', 84: '柳', 85: '拂', 86: '画堂', 87: '春', 88: '南北', 89: '东西', 90: '嗟', 91: '岁晚', 92: '路', 93: '白', 94: '同归', 95: '危桥', 96: '昔闻', 97: '洞庭', 98: '水', 99: '岳阳楼', 100: '苦乐', 101: '心由', 102: '我', 103: '

In [ ]:
# 添加 <START>, <END> 标记，并转为 tensor
sequences = []
max_len = 20

for poem in tokenized_poems:
    seq = [word2idx['<START>']] + tokenize(poem) + [word2idx['<END>']]
    if len(seq) <= max_len:
        sequences.append(torch.tensor(seq))

padded_sequences = pad_sequence(sequences, batch_first=True)

print("填充后的张量形状:", padded_sequences.shape)
print("填充后的张量:", padded_sequences)

填充后的张量形状: torch.Size([31072, 14])
填充后的张量: tensor([[   1,    4,    5,  ...,    0,    0,    0],
        [   1,   12,    3,  ...,    0,    0,    0],
        [   1,   15,   16,  ...,    0,    0,    0],
        ...,
        [   1,  505,  272,  ...,    0,    0,    0],
        [   1,  218, 1313,  ...,    0,    0,    0],
        [   1, 6120,  447,  ...,    0,    0,    0]])
